## Overview

- **Expectation Used:** [expect_profile_numeric_columns_diff_between_inclusive_threshold_range](https://github.com/great-expectations/great_expectations/blob/develop/contrib/capitalone_dataprofiler_expectations/capitalone_dataprofiler_expectations/expectations/expect_profile_numeric_columns_diff_between_inclusive_threshold_range.py)

- **Expectation Description:** This expectation will check the use-specified difference between use-specified metrics from an original report generated by the Data Profiler and a new report. The two reports are generated on two different sets of data with matching schemas.

- **Example Details:** In this example a data owner has a dataset containing salary information on individuals in the data science field ranging from **2020** to **2022**. The data owner has been tasked with checking the historical trends of salary growth across the industry with the numbers they have. They want to use this expectation to determine if the median growth of salaries at their company are within the same range of median growth as the companies they have data on.

### Imports

In [1]:
import os

import pandas as pd
import numpy as np

# Great expectations imports
import great_expectations as ge
from capitalone_dataprofiler_expectations.expectations. \
    expect_profile_numeric_columns_diff_between_inclusive_threshold_range \
    import ExpectProfileNumericColumnsDiffBetweenInclusiveThresholdRange
from great_expectations.self_check.util import build_pandas_validator_with_data

# Data Profiler import
import dataprofiler as dp

/Users/CXE445/Documents/DataProfiler/venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/CXE445/Documents/DataProfiler/venv/lib/python3.8/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/Users/CXE445/Documents/DataProfiler/venv/lib/python3.8/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/CXE445/Documents/DataProfiler/venv/lib/python3.8/site-packages/keras/utils/image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/Users/CXE445/Documents/D

INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 


  0%|          | 0/3 [00:00<?, ?it/s]/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.loc[true_sample_list]
 33%|███▎      | 1/3 [00:00<00:01,  1.17it/s]/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.loc[true_sample_list]
 67%|██████▋   | 2/3 [00:01<00:00,  1.97it/s]/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.loc[true_sample_list]
100%|██████████| 3/3 [00:01<00:00,  2.15it/s]

INFO:DataProfiler.profilers.profile_builder: Calculating the statistics... 



100%|██████████| 3/3 [00:00<00:00,  8.35it/s]
/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/numerical_column_stats.py:1597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(val, np.float):


...


100%|██████████| 5/5 [00:00<00:00, 12.02it/s]


INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 


  0%|          | 0/5 [00:00<?, ?it/s]/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.loc[true_sample_list]
 20%|██        | 1/5 [00:00<00:00,  4.15it/s]/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.loc[true_sample_list]
 40%|████      | 2/5 [00:00<00:00,  4.16it/s]/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.loc[true_sample_list]
 60%|██████    | 3/5 [00:00<00:00,  4.11it/s]/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: 

INFO:DataProfiler.profilers.profile_builder: Calculating the statistics... 



100%|██████████| 5/5 [00:00<00:00, 13.44it/s]


### Setup
Below we are going to import a dataset from the Data Profile testing suite. This csv holds information on the salaries of individuals in the data science field from all over the world.

In [2]:
context = ge.get_context()

In [3]:
data_path = "../../dataprofiler/tests/data/csv/ds_salaries.csv"
data = dp.Data(data_path).data
data

,uuid,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0c898791-6efc-4410-983d-c1346e1cb390,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,bf898c23-1c84-441f-addf-9cbf1fc16a90,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,e7ccbded-d7f8-470e-a5d0-45c84913c51e,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,ac158555-0bb5-48c4-927a-22984062adeb,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4be685e7-12c5-4171-8a19-4091b7f20699,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,87eb57bd-5289-407c-9158-f98bda3bfe67,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,b47f67a7-7803-4d91-ad7e-1909e6819571,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,5ecb8690-fbf3-42ed-baf5-41971988670b,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,99db8ec0-166c-4fd2-b4dd-a171221b41af,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


For this expectation we are going to compare the diff median `salary_in_usd` between `work_year` **2020** and **2022**. Below we are gathering the different years that are recorded in this dataset.

In [4]:
data.sort_values(by="work_year", axis=0, inplace=True)
years = data["work_year"].unique().tolist()
years

['2020', '2021', '2022']

Now that we have the years, we will capture all records from each year in their own dataframes so that we can process them separately.

In [5]:
individual_dataframes = []
for year in years:
    current_year_df = data.loc[data["work_year"]==year]
    current_year_df = current_year_df.drop("work_year", axis=1)
    individual_dataframes.append(current_year_df)
individual_dataframes[0]

,uuid,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0c898791-6efc-4410-983d-c1346e1cb390,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
51,715fb124-30a9-408e-831a-aa5d6d6e1c9e,EN,FT,Data Analyst,91000,USD,91000,US,100,US,L
50,2f6505ef-1d37-4cd9-894d-6ee6adf0745c,EN,FT,Data Analyst,450000,INR,6072,IN,0,IN,S
49,eb7a27be-3112-453b-9717-737e57f531ad,MI,FT,Data Engineer,61500,EUR,70139,FR,50,FR,L
48,4cb2acbd-f98f-40f5-ae52-626f65a726a1,MI,FT,Data Scientist,105000,USD,105000,US,100,US,L
...,...,...,...,...,...,...,...,...,...,...,...
23,f12ee8c6-7341-4357-9677-db9109a8194c,MI,FT,BI Data Analyst,98000,USD,98000,US,0,US,M
22,26c06feb-5914-4eed-86bb-e6f4187418ce,SE,FT,Data Engineer,42000,EUR,47899,GR,50,GR,L
21,038fb39d-42ef-4b1b-875c-5377b3540377,MI,FT,Product Data Analyst,450000,INR,6072,IN,100,IN,L
25,f29c47c7-620d-4ab5-9210-cd6ab57e7a76,EX,FT,Director of Data Science,325000,USD,325000,US,100,US,L


Now we will create a report on the first individual dataframe which corresponds to the year **2020**, then we will output the median `salary_in_usd` from this dataframe.

In [6]:
profiler_options = dp.ProfilerOptions()
profiler_options.set({"data_labeler.is_enabled": False})

profile = dp.Profiler(individual_dataframes[0], len(individual_dataframes[0]), options=profiler_options)
profile.save(filepath='previous_profile.pkl')
report  = profile.report(report_options={"output_format": "compact"})

INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns...  (with 11 processes)


100%|██████████| 11/11 [00:05<00:00,  1.88it/s]
/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.loc[true_sample_list]
/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.loc[true_sample_list]
/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.loc[true_sample_list]
/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.lo

INFO:DataProfiler.profilers.profile_builder: Calculating the statistics...  (with 4 processes)


100%|██████████| 11/11 [00:03<00:00,  3.34it/s]
/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/numerical_column_stats.py:1597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(val, np.float):


Let's take a look at the output `median` metric from the `salary_in_usd` column as found in the report

In [7]:
report['data_stats'][6]['statistics']['median']

75683.1475

The data owner has been told by his company that median salary increases were about **10000** from **2020** to **2022**. So the data owner wants to for their range around this number. With the validator below, an expectation is set up with a difference between the `median` `salary_in_usd` of at least **7500** and no more than **12500**. Meaning, this validator will check if the `median` `salary_in_usd` has increased by **7500** to **12500** from **2020 to **2022** across all the companies they have data on.

In [8]:
validator = build_pandas_validator_with_data(individual_dataframes[1])
results = validator.expect_profile_numeric_columns_diff_between_inclusive_threshold_range(
    profile_path='previous_profile.pkl',
    limit_check_report_keys={
            "salary_in_usd": {
                "median": {"lower": 7500, "upper": 12500},
            },
        }
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 



  0%|          | 0/11 [00:00<?, ?it/s]/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.loc[true_sample_list]
/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.loc[true_sample_list]
/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_series = df_series.loc[true_sample_list]

 27%|██▋       | 3/11 [00:00<00:00, 26.00it/s]/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/profile_builder.py:587: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a li

INFO:DataProfiler.profilers.profile_builder: Calculating the statistics... 




100%|██████████| 11/11 [00:00<00:00, 11.21it/s]
/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/utils.py:531: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mat1 = np.array(matrix1, dtype=np.float)
/Users/CXE445/Documents/DataProfiler/dataprofiler/profilers/utils.py:532: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mat2 = np

### Results
From the output below, the data owner can see that the expectation has an unexpected value. The result shows that the diff between the two profiles is slightly less than **7000**. This indicates to the data owner that the salary growth trends in competing companies are still below their own company.

In [9]:
results

{
  "success": false,
  "result": {
    "unexpected_values": {
      "salary_in_usd": {
        "median": {
          "lower_bound": 7500,
          "upper_bound": 12500,
          "value_found": 6968.818625
        }
      }
    }
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}